# Instacart Market Basket Analysis
### Which products will an Instacart consumer purchase again?

#### Data Introduction

The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order. The dataset is anonymized and contains a sample of over 3 million grocery orders from more than 200,000 Instacart users. For each user, we provide between 4 and 100 of their orders, with the sequence of products purchased in each order. We also provide the week and hour of day the order was placed, and a relative measure of time between orders. For more information, see the blog post accompanying its public release.
Blog post: https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2

In [2]:
import pandas as pd
import numpy as np

## Get Data

In [3]:
#aisles = pd.read_csv('db/aisles.csv')
# departments = pd.read_csv('db/departments.csv')
orders = pd.read_csv('db/orders.csv')
products = pd.read_csv('db/products.csv')
order_products_prior = pd.read_csv('db/order_products__prior.csv')

In [3]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [4]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [5]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [6]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [7]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [8]:
order_products_train = pd.read_csv('db/order_products__train.csv')
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [9]:
def unique_value_counts(df):
    for col in df.columns:
        print('Unique ', col, ': ', len(df[col].unique()))
        
def print_head(df):
    print(db.head())
    

def stats_accross_dbs(function_keyword):
    dbs = [aisles,departments,products,orders,order_products_prior]
    dbs_names = ['aisles','departments','products','orders','order_products_prior']

    for i in range(len(dbs)):
        print(dbs_names[i])
        db = dbs[i]
        
        if function_keyword == unique_value_counts:
            unique_value_counts(db)
       
        if function_keyword == print_head:
            print(db.head())
            
        if function_keyword == 'unique/len':
            unique_value_counts(db)
            print(len(db))
        
        #else: print('''function_keyword is not valid''')
        print('\n')
    

In [10]:
stats_accross_dbs('unique/len')

aisles
Unique  aisle_id :  134
Unique  aisle :  134
134


departments
Unique  department_id :  21
Unique  department :  21
21


products
Unique  product_id :  49688
Unique  product_name :  49688
Unique  aisle_id :  134
Unique  department_id :  21
49688


orders
Unique  order_id :  3421083
Unique  user_id :  206209
Unique  eval_set :  3
Unique  order_number :  100
Unique  order_dow :  7
Unique  order_hour_of_day :  24
Unique  days_since_prior_order :  32
3421083


order_products_prior
Unique  order_id :  3214874
Unique  product_id :  49677
Unique  add_to_cart_order :  145
Unique  reordered :  2
32434489




In [78]:
#stats_accross_dbs(print_head)

In [9]:
orders.columns

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')

In [67]:
orders.eval_set.unique()

array(['prior', 'train', 'test'], dtype=object)

In [68]:
orders.order_dow.unique()

array([2, 3, 4, 1, 5, 0, 6], dtype=int64)

In [11]:
orders_prior = orders[orders.eval_set == 'prior']
orders_train = orders[orders.eval_set == 'train']
orders_test = orders[orders.eval_set == 'test']

In [13]:
def get_unique_count(x):
    return len(np.unique(x))

cnt_srs = orders.groupby("eval_set")["user_id"].aggregate(get_unique_count)
cnt_srs

eval_set
prior    206209
test      75000
train    131209
Name: user_id, dtype: int64

In [19]:
cnt_srs = orders.groupby("eval_set")["user_id"].nunique()
cnt_srs

eval_set
prior    206209
test      75000
train    131209
Name: user_id, dtype: int64

In [20]:
orders.order_id.count()

3421083

In [22]:
orders.order_id.nunique()

3421083

## Features
##### Approches
**<p>1. % chance for the product to be sold</p>**
* Top products of the day
* Top Products by time

**<p>2. % chance for customer to buy a product</p>**

##### Questions
1) all order_id in order_products_prior exist in orders order_id?
2) check add_to_cart_order ==> what is it? ==> matches up with number of items === YES 


In [115]:
op = order_products_prior.order_id
o = orders.order_id

print('count op', len(op))
print('count o', len(o)) 
print('difference: ', len(op)-len(o),'\n')

print('unique count op', len(op.unique()))
print('unique count o', len(o.unique())) 
print('difference: ', len(op.unique())-len(o.unique()))

count op 32434489
count o 3421083
difference:  29013406 

unique count op 3214874
unique count o 3421083
difference:  -206209


In [ ]:
opu = order_products_prior.order_id.unique()
ou = orders.order_id.unique()

count = 0
for i in opu:
    if i not in ou:
        count+=1
print(count)

In [113]:
len(order_products_prior.order_id.unique())

3214874

## Feature Extraction

user_id | product_id | %orders_containing_product | reordered | add_to_cart_order | order_dow | order_hod | days_since_last_bought_the_item | aisle_id | department_id | orders_since_last bought_this_item

**item-wise features**
* avg_%orders_containing_product
* avarage_days_since_last_baught
* avg_add_to_cart_order

**user-wise features**
* avg_number_of_items
* avg_item_from_dept
* avg_item_from_aisle

%orders_containing_product = number_of_times_product_baught/total_number_of_orders

In [18]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [20]:
len(orders)

3421083

In [8]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [21]:
len(order_products_prior)

32434489

In [23]:
len(orders.order_id.unique())

3421083

In [9]:
df = pd.merge(order_products_prior, orders how='outer', on='order_id')

In [10]:
len(df)

32640698

In [11]:
df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196.0,1.0,0.0
1,2539329,1,prior,1,2,8,NaN,14084.0,2.0,0.0
2,2539329,1,prior,1,2,8,NaN,12427.0,3.0,0.0
3,2539329,1,prior,1,2,8,NaN,26088.0,4.0,0.0
4,2539329,1,prior,1,2,8,NaN,26405.0,5.0,0.0


Add number of orders

In [5]:
number_of_orders_by_users = orders.groupby(['user_id']).order_number.count().reset_index()

In [6]:
number_of_orders_by_users.head(3)

,user_id,order_number
0,1,11
1,2,15
2,3,13


In [ ]:
len(pd.merge(df, ))

In [15]:
len(df[df.user_id==3])

89

In [19]:
df2 = df[df.user_id==3]

In [20]:
df2

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
256,1374495,3,prior,1,1,14,NaN,9387.0,1.0,0.0
257,1374495,3,prior,1,1,14,NaN,17668.0,2.0,0.0
258,1374495,3,prior,1,1,14,NaN,15143.0,3.0,0.0
259,1374495,3,prior,1,1,14,NaN,16797.0,4.0,0.0
260,1374495,3,prior,1,1,14,NaN,39190.0,5.0,0.0
261,1374495,3,prior,1,1,14,NaN,47766.0,6.0,0.0
262,1374495,3,prior,1,1,14,NaN,21903.0,7.0,0.0
263,1374495,3,prior,1,1,14,NaN,39922.0,8.0,0.0
264,1374495,3,prior,1,1,14,NaN,24810.0,9.0,0.0
265,1374495,3,prior,1,1,14,NaN,32402.0,10.0,0.0


In [29]:
df2.groupby(['user_id']).count()

,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
user_id,,,,,,,,
3,167568463,541,94,1454,907.0,2416413.0,391.0,55.0


In [26]:
df2.groupby(['user_id']).count().columns

Index(['order_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered'],
      dtype='object')

In [39]:
def x(para):
    return 5

df2.groupby(['user_id', 'product_id']).agg(
    {'reordered': 'sum',
     'days_since_prior_order': 'mean', 
     'order_dow' : 'mean', # mode -- see unique values and frequency -- use apply
     'order_'
    })

reordered  days_since_prior_order  order_dow
user_id product_id                                              
3       248.0             0.0                9.000000   3.000000
        1005.0            0.0               17.000000   3.000000
        1819.0            2.0               11.333333   0.666667
        7503.0            0.0               21.000000   3.000000
        8021.0            0.0                9.000000   3.000000
        9387.0            4.0               11.500000   0.600000
        12845.0           0.0               20.000000   2.000000
        14992.0           1.0                7.000000   0.000000
        15143.0           0.0                     NaN   1.000000
        16797.0           2.0                7.000000   0.333333
        16965.0           1.0               16.000000   1.000000
        17668.0           4.0               12.250000   1.400000
        18370.0           0.0               12.000000   0.000000
        18599.0           3.0               11.500000   1.000000
        21137.0           0.0                9.000000   3.000000
        21903.0           7.0               11.714286   1.000000
        22035.0           2.0                8.666667   0.000000
        23650.0           1.0               12.000000   2.000000
        24010.0           1.0               13.500000   1.000000
        24810.0           2.0               11.000000   0.666667
        28373.0           1.0               16.500000   1.500000
        32402.0           2.0               13.000000   2.333333
        38596.0           0.0                9.000000   3.000000
        39190.0           9.0               11.555556   0.800000
        39922.0           0.0                     NaN   1.000000
        40604.0           0.0                9.000000   3.000000
        42265.0           0.0               12.000000   0.000000
        42557.0           0.0                7.000000   0.000000
        43961.0           3.0                8.000000   0.000000
        44683.0           1.0                9.500000   0.000000
        47766.0           8.0               12.125000   0.888889
        48523.0           1.0                7.000000   0.000000
        49683.0           0.0               21.000000   3.000000

In [22]:
len(df2.product_id.unique())

34

In [23]:
len(df2.order_id.unique())

13

In [ ]:
# How many items does each user buy? how many items do each user buy most frequently?

In [4]:
product_count_by_user = df.groupby(['user_id', 'product_id']).order_id.count()

NameError: name 'df' is not defined

In [18]:
product_count_by_user.plot(kind='hist')

KeyboardInterrupt: 

In [ ]:
# how to use different aggregation function when using groupby in pandas dataframe